In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
#from sklearn.metrics import mean_squared_error
#from sklearn.model_selection import KFold
#from sklearn.model_selection import cross_val_score

In [ ]:
X = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/train.csv', index_col=0)
X_test_full = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/test.csv" , index_col=0)

In [ ]:
X.fillna(X.std(), inplace = True)
X_test_full.fillna(X_test_full.std(), inplace = True)

In [ ]:
y = X.claim  
X.drop(['claim'], axis=1, inplace=True)

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
from sklearn.preprocessing import MinMaxScaler 
scaler = MinMaxScaler()
SS_X = scaler.fit_transform(X)
SS_X_test_full = scaler.transform(X_test_full)

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [ ]:
ordered_rank_features = SelectKBest(score_func = chi2, k = 114)
ordered_features = ordered_rank_features.fit_transform(SS_X,y)

In [ ]:
ordered_features.shape

In [ ]:
#dfScores = pd.DataFrame(ordered_features.scores_, columns = ["Score"])
#dfColumns = pd.DataFrame(X.columns)

In [ ]:
#features_rank = pd.concat([dfColumns,dfScores], axis = 1)
#features_rank.columns = ["Features", "Scores"]
#features_rank

In [ ]:
#features_rank.nlargest(60, "Scores")

In [ ]:
#selected_features = features_rank[features_rank["Scores"] > 20]
#print(len(selected_features))
#print(selected_features)

In [ ]:
X_train_full, X_valid_full, y_train, y_valid = train_test_split(ordered_features, y, test_size=0.30,
                                                                random_state=42,shuffle=True, stratify=y)

In [ ]:
from xgboost import XGBRegressor
XGmodel = XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.016,
                max_depth = 10, alpha = 10, n_estimators = 10)
XGmodel.fit(X_train_full, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

kfold = KFold(n_splits=10, random_state=7)
results = cross_val_score(XGmodel, X_train_full, y_train, cv=kfold)

In [ ]:
y_test_pred = XGmodel.predict(X_valid_full)

In [ ]:
mse = mean_squared_error(y_test_pred, y_valid)
mse

In [ ]:
SS_X_test_full = ordered_rank_features.transform(SS_X_test_full)

In [ ]:
df = pd.DataFrame(X_test_full)
df.head(3)

In [ ]:
y_test_pred = XGmodel.predict(SS_X_test_full)
y_test_pred

In [ ]:
output = pd.DataFrame({'Id': df.index,
                       'claim': y_test_pred})
output.to_csv('submission_13.csv', index = False)